# Sem Tech Project Gruppe 1

## Get whole Company List

In [48]:
# pip install sparqlwrapper
# pip install pandas
# https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

### Die aktuelle Query, welche zu jeder Company auch noch die SubCompanies liefert

query="""SELECT ?company ?besteht_aus WHERE {
  ?company wdt:P31 wd:Q4830453.
  OPTIONAL { ?company wdt:P1830 ?besteht_aus. }
}"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

results = get_results(endpoint_url, query)

# Länge der Liste
print(len(results["results"]["bindings"]))

for result in results["results"]["bindings"][:10]:
    print(result)
    pass

206171
{'company': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q66'}, 'besteht_aus': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q8793'}}
{'company': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q66'}, 'besteht_aus': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q374451'}}
{'company': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q66'}, 'besteht_aus': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q570930'}}
{'company': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q66'}, 'besteht_aus': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q795324'}}
{'company': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q66'}, 'besteht_aus': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q890212'}}
{'company': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q66'}, 'besteht_aus': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q961568'}}
{'company': {'type': 'uri', 'value': 'htt

## Define Company Object Class

In [49]:
class CompanyObject:
    def __init__(self, englishName, list_Languages, list_Lables, list_Aliases, 
                 TwitterUsername, TwitterID, jurisdiction, language, compID=0):
        
        self.compID=compID
        self.englishName=englishName
        self.list_Languages=list_Languages
        self.list_Lables=list_Lables
        self.list_Aliases=list_Aliases
        self.isSubsidiaryOf=isSubsidiaryOf

# Crawl HTML Code

In [50]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd


### First Tests of HTML Extraction

response = requests.get(
	url="https://www.wikidata.org/wiki/Q20716",
)
soup = BeautifulSoup(response.content, 'html.parser')

languages=[]
lables=[]

table = soup.find_all("tr", class_=re.compile("wikibase-entitytermsforlanguageview"))

englishName=table[0].parent.find("div").find("span").contents[0]


for elem in table:
    parentElement=elem.parent
    # Add Language Name to the Array
    languages.append(parentElement.find("th").contents[0])
    # Add Label to the other Array
    lables.append(parentElement.find("div").find("span").contents[0])
    print("I found Language: ",parentElement.find("th").contents[0])

I found Language:  English


# Crawl Json

In [51]:
import requests, json
from bs4 import BeautifulSoup
import re
import pandas as pd

results = get_results(endpoint_url, query)

def crawl(compID):
    response = requests.get(
        url=("https://www.wikidata.org/wiki/Special:EntityData/"+compID+".json"))
    data = json.loads(response.text)
    df = pd.DataFrame(data['entities'][compID]['labels'])
    
    languages=df.iloc[0]
    labels=df.iloc[1]
    
#   df = pd.DataFrame(data['entities'][compID]['aliases'])
    
#   aliases = (df.iloc[0],df.iloc[1])
    
### TODO: Get further Data from HTML
    
     
#    compObject = CompanyObject('Test',
#                             languages, labels, aliases
#                             )


crawl('Q1418')
# Länge der Liste
#print(len(results["results"]["bindings"]))

for result in results["results"]["bindings"][:10]:
    pass

In [52]:
### Example Instances

# Example Instance of Mother Company
NokiaMother = CompanyObject('Nokia',['Englisch','Japan'],['Nokia','JapanNokia'],'AG','More than 1000',
                     'More than 1 Billion','Nok_CEO','Nok_CIO', 'null')

# How to Access it
print("Name: ",NokiaMother.englishName,"\n")

# Example Instance of Daughter Company
NokiaDaughter = CompanyObject('Nokia_Daughter',['Englisch','Japan'],['NokiaDaugh','JapanNokiaDaugh'],'AG','More than 10',
                               'More than 1 Million','Nok_CEODaugh','Nok_CIODaugh', NokiaMother)

# How to Access it # Access to MotherCompany( if available)
print("Name: ",NokiaDaughter.englishName," is a subsidiary of: ",NokiaDaughter.isSubsidiaryOf.englishName)

NameError: name 'isSubsidiaryOf' is not defined

# Setting up the ontology

In [53]:
from owlready2 import *

#load the basic structure of the ontology that was created in webprotege
onto = get_ontology("file://names_onto.owl").load()

classes = []

#extract the classes so we can create instances
for cl in onto.classes():
    classes.append(cl)



# Basic functions to create instances in order to fill our ontology

In [54]:
#TODO add attributes for instances when needed. Run for all aquired data.

def get_Company_Instance(name):
    cl = [c for c in classes if c.name == "Company"]
    return cl[0](name)

def get_Alias_Instance(name):
    cl = [c for c in classes if c.name == "Alias"]
    return cl[0](name)

def get_Language_Instance(name):
    cl = [c for c in classes if c.name == "Language"]
    return cl[0](name)

def get_Dialect_Instance(name):
    cl = [c for c in classes if c.name == "Dialect"]
    return cl[0](name)

def get_Name_Instance(name):
    cl = [c for c in classes if c.name == "Name"]
    return cl[0](name)

def get_TikTok_Instance(name):
    cl = [c for c in classes if c.name == "TikTik_Username"]
    return cl[0](name)

def get_Twitter_Instance(name):
    cl = [c for c in classes if c.name == "Twitter_Username"]
    return cl[0](name)
    
def get_Youtube_Instance(name):
    cl = [c for c in classes if c.name == "Youtube_Username"]
    return cl[0](name)

def get_SubCompany_Instance(name):
    cl = [c for c in classes if c.name == "Subsidiary_company"]
    return cl[0](name)



In [55]:
#never run this again unless absolutely necessary, takes a while..

values = []
owned_by = []
companies = []

for company in results["results"]["bindings"]:
    entity_number = company["company"]["value"][31:] 
    try: 
        sub_number = company["besteht_aus"]["value"][31:]  
        owned_by.append({"owner": entity_number, "owned": sub_number})
    except KeyError:
        pass
    values.append(entity_number)

for value in values:
    company = get_Company_Instance(value)
    companies.append(company)


for pair in owned_by:
    if pair["owner"] in values and pair["owned"] in values:
        owner = [c for c in companies if c.name == pair["owner"]][0]
        owned = [c for c in companies if c.name == pair["owned"]][0]
        owner.is_parent_company_of.append(owned)



# Example on how to link created instances via properties

In [56]:

"""
alias = get_Alias_Instance("Alias")

Name = get_Name_Instance("Name")

alias.belongs_to.append(Name)
"""


'\nalias = get_Alias_Instance("Alias")\n\nName = get_Name_Instance("Name")\n\nalias.belongs_to.append(Name)\n'

# Some sample querries running on our ontology

In [57]:
print("Number of classes:", list(default_world.sparql("""
           SELECT (COUNT(?x) AS ?nb)
           { ?x a owl:Class . }
    """)))

print("One example class", list(default_world.sparql("""
           SELECT ?x
           { ?x rdfs:label "Alias" . }
    """)))

print("All classes with their sub-classes:", list((default_world.sparql("""
           SELECT ?x ?y
           { ?x a owl:Class . 
             ?x rdfs:subClassOf* ?y }
    """))))


Number of classes: [[9]]
One example class [[names_onto.Alias]]
All classes with their sub-classes: [[names_onto.Company, names_onto.Company], [names_onto.Company, owl.Thing], [names_onto.Name, names_onto.Name], [names_onto.Name, owl.Thing], [names_onto.Alias, names_onto.Alias], [names_onto.Alias, names_onto.Name], [names_onto.Alias, owl.Thing], [names_onto.Dialect, names_onto.Dialect], [names_onto.Dialect, names_onto.Language], [names_onto.Dialect, owl.Thing], [names_onto.Subsidiary_company, names_onto.Subsidiary_company], [names_onto.Subsidiary_company, names_onto.Company], [names_onto.Subsidiary_company, owl.Thing], [names_onto.Language, names_onto.Language], [names_onto.Language, owl.Thing], [names_onto.TikTok_Username, names_onto.TikTok_Username], [names_onto.TikTok_Username, names_onto.Name], [names_onto.TikTok_Username, owl.Thing], [names_onto.Twitter_Username, names_onto.Twitter_Username], [names_onto.Twitter_Username, names_onto.Name], [names_onto.Twitter_Username, owl.Thing],

# Using sync_reasoner to find and fix inconsistencies 

In [58]:
#automatically finds inconsistencies and reclassifies individuals and superclasses
#especially helpful when circular dependencies are present 
sync_reasoner()


#optional: use diffrent reasoner, no differences spotted yet
#sync_reasoner_pellet() 


#optional: also taking object property values into acount
#sync_reasoner(infer_property_values = True)

* Owlready2 * Running HermiT...
    java -Xmx2000M -cp /Users/leonhardziegler/Library/Mobile Documents/com~apple~CloudDocs/UniBw/Master/SematischeTech/Übungen/Projekt/SemTechProject/ontoEnv/lib/python3.8/site-packages/owlready2/hermit:/Users/leonhardziegler/Library/Mobile Documents/com~apple~CloudDocs/UniBw/Master/SematischeTech/Übungen/Projekt/SemTechProject/ontoEnv/lib/python3.8/site-packages/owlready2/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////var/folders/d8/_r05917j3p57_y1gpng5x1rr0000gn/T/tmpwviy9kvl
* Owlready2 * HermiT took 9.311551094055176 seconds
* Owlready * Reparenting names_onto.Q51450967: {names_onto.Company} => {names_onto.Subsidiary_company}
* Owlready * Reparenting names_onto.Q40650: {names_onto.Company} => {names_onto.Subsidiary_company}
* Owlready * Reparenting names_onto.Q5013473: {names_onto.Company} => {names_onto.Subsidiary_company}
* Owlready * Reparenting names_onto.Q5013497: {names_onto.Company} => {names_onto.Subsidiary_co

# Saving changes

In [60]:
# weird: owlready2 cannot overwrite a file passed into the function via the first parameter. It can only create a new files...

onto.save("filled_names_onto4", "rdfxml")